# Memory (대화 내용 기억)
`04_memory.ipynb`
- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다(stateless)
- 이전 대화내용을 계속 프롬프트에 주입해야 함

1. Short-Term Memory
    - 단기기억: 한 대화 세션에 대한 기억
2. Long-Term Memory
    - 장기기억: 전체 세션에서 추출한 중요한 정보

## Memory 구동 방식
1. 메모리는 기본적으로 모든 대화 내역을 LLM Input에 밀어 넣는것.
2. 이때 대화가 길어지면 토큰수 증가 및 성능 하락이 일어남
3. 개선 방식 컨셉
    1. 요약
    2. 적정 길이에서 앞부분 자르기
    3. 정리(특정 명사들로 정리, Node-Edge 그래프 방식 등)

## `ConversationBufferMemory`
- 메시지 저장 -> 변수에서 추출 가능

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})

In [ ]:
from operator import itemgetter
# `chat_history`변수에, load_memory_var 결과를 저장하고, `chat_history 키를 추출`
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

In [ ]:
chain = runnable | prompt | llm

In [ ]:
res = chain.invoke({'input': '만나서 반가워'})

In [ ]:
memory.save_context(
    {'human': '만나서 반가워'},
    {'ai': res.content}
)

In [ ]:
my_message = '내 이름 뭐라고?'
res = chain.invoke({'input': my_message})
memory.save_context(
    {'human': my_message},
    {'ai': res.content}
)

In [ ]:
memory.load_memory_variables({})

In [28]:
import time
# 실제 gpt랑 대화하는 모습으로 만들기

input_msg = ''

# 사용자가 ('quit', '정지', '그만') 중에 하나를 입력하면 대화 종료
while input_msg not in ('quit', '정지', '그만'):
    input_msg = input()
    # 대화 로직 + 대화내역 아래 출력
    print('인간: ', input_msg)
    print('AI: ', 'AI의 답변...')
    time.sleep(0.1)


인간:  하이요
AI:  AI의 답변...
인간:  내 이름은 뭐야
AI:  AI의 답변...
인간:  내이름 뭐야?
AI:  AI의 답변...
인간:  정지
AI:  AI의 답변...
